# 导入工具包

In [1]:
# opencv-python
import cv2
import numpy as np
# mediapipe人工智能工具包
import mediapipe as mp
# 进度条库
from tqdm import tqdm
# 时间库
import time

# 导入模型

In [2]:
# 导入三维人脸关键点检测模型
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(   
        static_image_mode=False,      # 是静态图片还是连续视频帧，摄像头画面为连续视频帧，此处选False
        refine_landmarks=True,       # 使用注意力机制Attention Mesh Model，对嘴唇、眼睛、瞳孔周围的关键点精细定位
        max_num_faces=5,              # 最多检测几张脸
        min_detection_confidence=0.5, # 置信度阈值
        min_tracking_confidence=0.5,  # 追踪阈值
)

# 导入可视化绘图函数
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=[66,77,229])

# 处理单帧的函数

In [3]:
# 处理帧函数
def process_frame(img):
    
    # 记录该帧开始处理的时间
    start_time = time.time()

    # 获取图像宽高
    h,w = img.shape[0], img.shape[1]
    # BGR转RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 将RGB图像输入模型，获取预测结果
    results = model.process(img_RGB)

    # 可视化参数
    scaler = 1 # 字体大小
    radius = 5 # 关键点圆的半径
    lw = 2 # 直线线宽

    if results.multi_face_landmarks: # 如果检测出人脸
        
        # 可视化468个关键点及对应的编号
        for face_landmarks in results.multi_face_landmarks: # 遍历每一张脸
            mp_drawing.draw_landmarks(
                  image=img,
                  landmark_list=face_landmarks,
                  connections=mp_face_mesh.FACEMESH_CONTOURS,
                  landmark_drawing_spec=drawing_spec,
                  connection_drawing_spec=drawing_spec)
            for idx, coord in enumerate(face_landmarks.landmark): # 遍历每一个关键点
                cx = int(coord.x * w)
                cy = int(coord.y * h)
                # 图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
                img = cv2.putText(img, str(idx), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.2*scaler, (0, 255, 0), 1)

        # 获取相关关键点坐标
        
        # 脸轮廓四周边缘点
        # 脸上边缘
        FT = results.multi_face_landmarks[0].landmark[10]; FT_X, FT_Y = int(FT.x * w), int(FT.y * h); FT_Color = (31,41,81)
        # img = cv2.circle(img,(FT_X, FT_Y), radius, FT_Color, -1)
        # 脸轮廓最左点
        FL = results.multi_face_landmarks[0].landmark[234]; FL_X, FL_Y = int(FL.x * w), int(FL.y * h); FL_Color = (0,0,255)
        # img = cv2.circle(img,(FL_X, FL_Y), radius, FL_Color, -1)
        # 脸轮廓最下边缘
        FB = results.multi_face_landmarks[0].landmark[152]; FB_X, FB_Y = int(FB.x * w), int(FB.y * h); FB_Color = (31,41,81)
        # img = cv2.circle(img,(FB_X, FB_Y), radius, FB_Color, -1)
        # 脸轮廓最右点
        FR = results.multi_face_landmarks[0].landmark[454]; FR_X, FR_Y = int(FR.x * w), int(FR.y * h); FR_Color = (0,255,0)
        # img = cv2.circle(img,(FR_X, FR_Y), radius, FR_Color, -1)
         
        # 五眼
        # 左边眼睛左眼角
        ELL = results.multi_face_landmarks[0].landmark[33]; ELL_X, ELL_Y = int(ELL.x * w), int(ELL.y * h); ELL_Color = (255,0,0)
        img = cv2.circle(img,(ELL_X, ELL_Y), radius, ELL_Color, -1)
        # 左边眼睛右眼角
        ELR = results.multi_face_landmarks[0].landmark[133]; ELR_X, ELR_Y = int(ELR.x * w), int(ELR.y * h); ELR_Color = (0,255,255)
        img = cv2.circle(img,(ELR_X, ELR_Y), radius, ELR_Color, -1)
        # 右边眼睛左眼角
        ERL = results.multi_face_landmarks[0].landmark[362]; ERL_X, ERL_Y = int(ERL.x * w), int(ERL.y * h); ERL_Color = (223,155,6)
        img = cv2.circle(img,(ERL_X, ERL_Y), radius, ERL_Color, -1)
        # 右边眼睛右眼角
        ERR = results.multi_face_landmarks[0].landmark[263]; ERR_X, ERR_Y = int(ERR.x * w), int(ERR.y * h); ERR_Color = (151,57,224)
        img = cv2.circle(img,(ERR_X, ERR_Y), radius, ERR_Color, -1)
        # 计算“五眼指标”
        # 从左往右六个点的横坐标
        Six_X = np.array([FL_X, ELL_X, ELR_X, ERL_X, ERR_X, FR_X])
        # 从最左到最右的距离
        Left_Right = FR_X - FL_X
        # 从左往右六个点间隔的五个距离，并归一化
        Five_Distance = 100 * np.diff(Six_X) / Left_Right
        # 两眼宽度的平均值
        Eye_Width_Mean = np.mean([Five_Distance[1], Five_Distance[3]])
        # 五个距离 与 两眼宽度均值 的差
        Five_Eye_Diff = Five_Distance - Eye_Width_Mean
        # 求L2范数，作为颜值的“五眼”评价指标
        Five_Eye_Metrics = np.linalg.norm(Five_Eye_Diff)
        
        # 三庭
        # 眉心
        MX = results.multi_face_landmarks[0].landmark[9]; MX_X, MX_Y = int(MX.x * w), int(MX.y * h); MX_Color = (29,123,243)
        img = cv2.circle(img,(MX_X, MX_Y), radius, MX_Color, -1)
        # 鼻翼下缘
        NB = results.multi_face_landmarks[0].landmark[2]; NB_X, NB_Y = int(NB.x * w), int(NB.y * h); NB_Color = (180,187,28)
        img = cv2.circle(img,(NB_X, NB_Y), radius, NB_Color, -1)
        # 嘴唇中心
        LC = results.multi_face_landmarks[0].landmark[13]; LC_X, LC_Y = int(LC.x * w), int(LC.y * h); LC_Color = (0,0,255)
        img = cv2.circle(img,(LC_X, LC_Y), radius, LC_Color, -1)
        # 嘴唇下缘
        LB = results.multi_face_landmarks[0].landmark[17]; LB_X, LB_Y = int(LB.x * w), int(LB.y * h); LB_Color = (139,0,0)
        img = cv2.circle(img,(LB_X, LB_Y), radius, LB_Color, -1)
        # 从上到下六个点
        Six_Y = np.array([FT_Y, MX_Y, NB_Y, LC_Y, LB_Y, FB_Y])
        # 从最上到最下的距离
        Top_Down = FB_Y - FT_Y
        # 从上到下六个点间隔的五个距离，并归一化
        Three_Section_Distance = 100 * np.diff(Six_Y) / Top_Down
        # 三庭的后两庭是否接近，越小越好
        Three_Section_Metric_A = np.abs(Three_Section_Distance[1] - sum(Three_Section_Distance[2:]))
        # 鼻下到唇心距离 占 第三庭的三分之一
        Three_Section_Metric_B = np.abs(Three_Section_Distance[2] - sum(Three_Section_Distance[2:])/3)
        # 唇心到下巴尖距离 占 第三庭的二分之一
        Three_Section_Metric_C = np.abs(sum(Three_Section_Distance[3:]) - sum(Three_Section_Distance[2:])/2)
        
        # 达芬奇
        # 嘴唇左角
        LL = results.multi_face_landmarks[0].landmark[61]; LL_X, LL_Y = int(LL.x * w), int(LL.y * h); LL_Color = (255,255,255)
        img = cv2.circle(img,(LL_X, LL_Y), radius, LL_Color, -1)
        # 嘴唇右角
        LR = results.multi_face_landmarks[0].landmark[291]; LR_X, LR_Y = int(LR.x * w), int(LR.y * h); LR_Color = (255,255,255)
        img = cv2.circle(img,(LR_X, LR_Y), radius, LR_Color, -1)
        # 鼻子左缘
        NL = results.multi_face_landmarks[0].landmark[129]; NL_X, NL_Y = int(NL.x * w), int(NL.y * h); NL_Color = (255,255,255)
        img = cv2.circle(img,(NL_X, NL_Y), radius, NL_Color, -1)
        # 鼻子右缘
        NR = results.multi_face_landmarks[0].landmark[358]; NR_X, NR_Y = int(NR.x * w), int(NR.y * h); NR_Color = (255,255,255)
        img = cv2.circle(img,(NR_X, NR_Y), radius, NR_Color, -1)
        # 嘴宽为鼻宽的1.5-1.6倍
        Da_Vinci = (LR.x - LL.x) / (NR.x - NL.x)
        
        # 眉毛
        # 左眉毛左眉角：46
        EBLL = results.multi_face_landmarks[0].landmark[46]; EBLL_X, EBLL_Y = int(EBLL.x * w), int(EBLL.y * h); EBLL_Color = (0,255,255)
        img = cv2.circle(img,(EBLL_X, EBLL_Y), radius, EBLL_Color, -1)
        # 左眉毛眉峰：105
        EBLT = results.multi_face_landmarks[0].landmark[105]; EBLT_X, EBLT_Y = int(EBLT.x * w), int(EBLT.y * h); EBLT_Color = (0,255,255)
        img = cv2.circle(img,(EBLT_X, EBLT_Y), radius, EBLT_Color, -1)
        # 左眉毛右角：107
        EBLR = results.multi_face_landmarks[0].landmark[107]; EBLR_X, EBLR_Y = int(EBLR.x * w), int(EBLR.y * h); EBLR_Color = (0,255,255)
        img = cv2.circle(img,(EBLR_X, EBLR_Y), radius, EBLR_Color, -1)
        # 右眉毛左角：336
        EBRL = results.multi_face_landmarks[0].landmark[336]; EBRL_X, EBRL_Y = int(EBRL.x * w), int(EBRL.y * h); EBRL_Color = (0,255,255)
        img = cv2.circle(img,(EBRL_X, EBRL_Y), radius, EBRL_Color, -1)
        # 右眉毛眉峰：334
        EBRT = results.multi_face_landmarks[0].landmark[334]; EBRT_X, EBRT_Y = int(EBRT.x * w), int(EBRT.y * h); EBRT_Color = (0,255,255)
        img = cv2.circle(img,(EBRT_X, EBRT_Y), radius, EBRT_Color, -1)
        # 右眉毛右眉角：276
        EBRR = results.multi_face_landmarks[0].landmark[276]; EBRR_X, EBRR_Y = int(EBRR.x * w), int(EBRR.y * h); EBRR_Color = (0,255,255)
        img = cv2.circle(img,(EBRR_X, EBRR_Y), radius, EBRR_Color, -1)
        # 内测眉头在内眦（内测眼角）正上方-左侧
        # 越接近0越好
        EB_Metric_A = (EBLR_X - ELR_X) / Left_Right
        # 内测眉头在内眦（内测眼角）正上方-右侧
        # 越接近0越好
        EB_Metric_B = (EBRL_X - ERL_X) / Left_Right
        # 眉峰在外眦（外测眼角）正上方-左侧
        # 越接近0越好
        EB_Metric_C = (EBLT_X - ELL_X) / Left_Right
        # 眉峰在外眦（外测眼角）正上方-右侧
        # 越接近0越好
        EB_Metric_D = (EBRT_X - ERR_X) / Left_Right
        # 外侧眉峰、外侧眼角、鼻翼 应 处于同一条直线上-左侧
        # 计算这三点构成的三角形面积，越小越好
        EB_Metric_E = 0.5 * np.linalg.det([[EBLL_X,EBLL_Y,1],[ELL_X,ELL_Y,1],[NL_X,NL_Y,1]]) / (Left_Right)**2
        # 外侧眉峰、外侧眼角、鼻翼 应 处于同一条直线上-右侧
        # 计算这三点构成的三角形面积，越小越好
        EB_Metric_F = 0.5 * np.linalg.det([[EBRR_X,EBRR_Y,1],[ERR_X,ERR_Y,1],[NR_X,NR_Y,1]]) / (Left_Right)**2
        # 可视化
        # 外侧眉峰、外侧眼角、鼻翼 应 处于同一条直线上-左侧
        cv2.line(img,(EBLL_X, EBLL_Y),(ELL_X, ELL_Y), EBLL_Color, lw)
        cv2.line(img,(ELL_X, ELL_Y),(NL_X, NL_Y), EBLL_Color, lw)
        cv2.line(img,(EBLL_X, EBLL_Y),(NL_X, NL_Y), EBLL_Color, lw)
        # 外侧眉峰、外侧眼角、鼻翼 应 处于同一条直线上-右侧
        cv2.line(img,(EBRR_X, EBRR_Y),(ERR_X, ERR_Y), EBLL_Color, lw)
        cv2.line(img,(ERR_X, ERR_Y),(NR_X, NR_Y), EBLL_Color, lw)
        cv2.line(img,(EBRR_X, EBRR_Y),(NR_X, NR_Y), EBLL_Color, lw)
        img = cv2.putText(img, 'Eyebrow', (25 * scaler, 500 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'A {:.2f} B {:.2f} C {:.2f} D {:.2f}'.format(EB_Metric_A,EB_Metric_B,EB_Metric_C,EB_Metric_D), (25 * scaler, 550 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'Eyebrow-Eye-Nose', (25 * scaler, 600 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'E {:.2f} F {:.2f}'.format(EB_Metric_E, EB_Metric_F), (25 * scaler, 650 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        
        # 内眼角
        # 左内眼角上点：157
        ELRT = results.multi_face_landmarks[0].landmark[157]; ELRT_X, ELRT_Y = int(ELRT.x * w), int(ELRT.y * h); ELRT_Color = (0,255,255)
        img = cv2.circle(img,(ELRT_X, ELRT_Y), radius, ELRT_Color, -1)
        # 左内眼角下点：154
        ELRB = results.multi_face_landmarks[0].landmark[154]; ELRB_X, ELRB_Y = int(ELRB.x * w), int(ELRB.y * h); ELRB_Color = (0,255,255)
        img = cv2.circle(img,(ELRB_X, ELRB_Y), radius, ELRB_Color, -1)
        # 右内眼角上点：384
        ERLT = results.multi_face_landmarks[0].landmark[384]; ERLT_X, ERLT_Y = int(ERLT.x * w), int(ERLT.y * h); ERLT_Color = (0,255,255)
        img = cv2.circle(img,(ERLT_X, ERLT_Y), radius, ERLT_Color, -1)
        # 右内眼角下点：381
        ERRB = results.multi_face_landmarks[0].landmark[381]; ERRB_X, ERRB_Y = int(ERRB.x * w), int(ERRB.y * h); ERRB_Color = (0,255,255)
        img = cv2.circle(img,(ERRB_X, ERRB_Y), radius, ERRB_Color, -1)
        # 内眼角开合度数-左侧 
        # 48度至50度为宜
        vector_a = np.array([ELRT_X-ELR_X,ELRT_Y-ELR_Y])
        vector_b = np.array([ELRB_X-ELR_X,ELRB_Y-ELR_Y])
        cos = vector_a.dot(vector_b)/(np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
        EB_Metric_G = np.degrees(np.arccos(cos))
        # 内眼角开合度数-右侧
        # 48度至50度为宜
        vector_a = np.array([ERLT_X-ERL_X,ERLT_Y-ERL_Y])
        vector_b = np.array([ERRB_X-ERL_X,ERRB_Y-ERL_Y])
        cos = vector_a.dot(vector_b)/(np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
        EB_Metric_H = np.degrees(np.arccos(cos))
        # 可视化
        cv2.line(img,(ELR_X, ELR_Y),(ELRT_X, ELRT_Y), ELRT_Color, lw) # 左侧内眦，眼角与内眼角上点连线
        cv2.line(img,(ELR_X, ELR_Y),(ELRB_X, ELRB_Y), ELRB_Color, lw) # 左侧内眦，眼角与内眼角下点连线
        cv2.line(img,(ERL_X, ELR_Y),(ERLT_X, ERLT_Y), ERLT_Color, lw) # 右侧内眦，眼角与内眼角上点连线
        cv2.line(img,(ERL_X, ELR_Y),(ERRB_X, ERRB_Y), ERLT_Color, lw) # 右侧内眦，眼角与内眼角下点连线
        img = cv2.putText(img, 'Inner Angle L:{:.2f} R:{:.2f}'.format(EB_Metric_G,EB_Metric_H), (25 * scaler, 700 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
        # 画五眼的竖线
        cv2.line(img,(FL_X, FT_Y),(FL_X, FB_Y), FL_Color, lw)
        cv2.line(img,(ELL_X, FT_Y),(ELL_X, FB_Y), ELL_Color, lw)
        cv2.line(img,(ELR_X, FT_Y),(ELR_X, FB_Y), ELR_Color, lw)
        cv2.line(img,(ERL_X, FT_Y),(ERL_X, FB_Y), ERL_Color, lw)
        cv2.line(img,(ERR_X, FT_Y),(ERR_X, FB_Y), ERR_Color, lw)
        cv2.line(img,(FR_X, FT_Y),(FR_X, FB_Y), FR_Color, lw)
        # 画脸上下边缘的横线
        cv2.line(img,(FL_X, FT_Y),(FR_X, FT_Y), FT_Color, lw)
        cv2.line(img,(FL_X, FB_Y),(FR_X, FB_Y), FB_Color, lw)
        # 画三庭的横线
        cv2.line(img,(FL_X, MX_Y),(FR_X, MX_Y), MX_Color, lw)
        cv2.line(img,(FL_X, NB_Y),(FR_X, NB_Y), NB_Color, lw)
        cv2.line(img,(FL_X, LC_Y),(FR_X, LC_Y), LC_Color, lw)
        cv2.line(img,(FL_X, LB_Y),(FR_X, LB_Y), LB_Color, lw)

        scaler = 1
        img = cv2.putText(img, 'Five Eye {:.2f}'.format(Five_Eye_Metrics), (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'A {:.2f}'.format(Five_Eye_Diff[0]), (25 * scaler, 150 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'B {:.2f}'.format(Five_Eye_Diff[2]), (25 * scaler, 200 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, 'C {:.2f}'.format(Five_Eye_Diff[4]), (25 * scaler, 250 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

        img = cv2.putText(img, 'Three Section {:.2f}'.format(Three_Section_Metric_A), (25 * scaler, 300 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, '1/3 {:.2f}'.format(Three_Section_Metric_B), (25 * scaler, 350 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        img = cv2.putText(img, '1/2 {:.2f}'.format(Three_Section_Metric_C), (25 * scaler, 400 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)

        img = cv2.putText(img, 'Da Vinci {:.2f}'.format(Da_Vinci), (25 * scaler, 450 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    else:
        img = cv2.putText(img, 'No Face Detected', (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)
    img = cv2.putText(img, 'FPS  '+str(int(FPS)), (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
    return img

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 调用摄像头获取每帧（模板）

In [4]:
# 调用摄像头逐帧实时处理模板
# 不需修改任何代码，只需修改process_frame函数即可

# 导入opencv-python
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头
cap = cv2.VideoCapture(1)

# 打开cap
cap.open(0)

# 无限循环，直到break被触发
while cap.isOpened():
    # 获取画面
    success, frame = cap.read()
    if not success:
        print('Error')
        break
    start_time = time.time()
    
    ## !!!处理帧函数
    frame = process_frame(frame)
    
    # 展示处理后的三通道图像
    cv2.imshow('my_window',frame)

    if cv2.waitKey(1) in [ord('q'),27]: # 按键盘上的q或esc退出（在英文输入法下）
        break
    
# 关闭摄像头
cap.release()

# 关闭图像窗口
cv2.destroyAllWindows()

[ WARN:0@0.691] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video1): can't open camera by index


如果遇到Error，需检查其它notebook是否占用了摄像头，需在其它notebook中restart kernel

# 视频逐帧处理（模板）

In [5]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义process_frame函数即可

def generate_video(input_path='./videos/three-hands.mp4'):
    filehead = input_path.split('/')[-1]
    output_path = "out-" + filehead
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    # cv2.namedWindow('Crack Detection and Measurement Video Processing')
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))
    # fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                # frame_path = './temp_frame.png'
                # cv2.imwrite(frame_path, frame)
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if success == True:
                    # cv2.imshow('Video Processing', frame)
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

                # if cv2.waitKey(1) & 0xFF == ord('q'):
                    # break
        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

In [6]:
generate_video(input_path='videos/single_person5_metrics.mp4')

视频开始处理 videos/single_person5_metrics.mp4
视频总帧数为 0


0it [00:00, ?it/s]

视频已保存 out-single_person5_metrics.mp4
